In [1]:
from llama_index.core.schema import NodeWithScore
class HybridSearch():
    def __init__(self, bm25_retriever, semantic_retriever,
                 bm25_weight=0.5, semantic_weight=0.5):
        self.bm25_retriever = bm25_retriever # bm25 검색기 객체
        self.semantic_retriever = semantic_retriever # 밀집 검색 객체
        self.bm25_weight = bm25_weight # bm25 가중치
        self.semantic_weight = semantic_weight # 밀집 검색 가중치

    def _get_bm25_retrieve(self, query:str)->list:
        result = self.bm25_retriever.retrieve(query)
        # (node 객체, 가중치 반영 점수)의 튜플 리스트로 반환
        score = [(node.node, node.score * self.bm25_weight) for node in result]
        return score

    def _get_semantic_retrieve(self, query:str)->list:
        result = self.semantic_retriever.retrieve(query)
        # (node 객체, 가중치 반영 점수)의 튜플 리스트로 반환
        score = [(node.node, node.score * self.semantic_weight) for node in result]
        return score

    # 혼합 검색 node 반환
    def retrieve(self, query:str)->list[NodeWithScore]:
        bm25_result = self._get_bm25_retrieve(query=query)
        semantic_result = self._get_semantic_retrieve(query=query)
        
        # key: node 객체, value: 최종 점수
        combined_scores = {}

        # BM25 점수 반영
        for node, score in bm25_result:
            combined_scores[node.text] = {'node':node, 'score':score}
        
        # Semantic 점수 반영
        for node, score in semantic_result:
            if node.text in combined_scores: # 이미 존재하는 청크인 경우
                combined_scores[node.text]['score'] += score
            else: # 새로운 청크인 경우
                
                combined_scores[node.text] = {'node':node, 'score':score}
        
        # NodeWithScore 형태 리스트로 변환
        final_results = []
        for _, node_info in combined_scores.items():
            if node_info['score'] == 0: continue # 점수가 0점인 경우 제외
            final_results.append(NodeWithScore(node=node_info['node'], score=node_info['score']))
        return final_results

In [2]:
### Step1. 문서 준비
sample_documents = [
    "RAG 기술의 최신 동향",
    "최신 검색 증강 시스템 연구",
    "RAG을 활용한 챗봇 개발",
    "RAG와 Dense Passage Retrieval의 비교 연구",
    "최신 AI 논문: 검색 증강 기술",
]

### Document 객체 생성
from llama_index.core import Document
documents = [Document(text=doc) for doc in sample_documents]

In [3]:
### Step2. 희소 검색을 위한 형태소 분석기 함수 작성
from konlpy.tag import Okt
okt = Okt() # Okt 형태소 분석기 초기화
def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
    return okt.morphs(text)  # 한국어 형태소 기반 토큰화

In [4]:
### Step3-1 API KEY 설정
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
### Step3-2.  LLM 및 Embedding Model 설정
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)  # 모델명은 예시
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [6]:
### Step4. 청킹을 위한 Splitter 설정
# 청킹 (chunking)
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [7]:
### 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context1 = StorageContext.from_defaults(persist_dir="./index/ch03_hybrid_search_keyword_storage") 
storage_context2 = StorageContext.from_defaults(persist_dir="./index/ch03_hybrid_search_vertor_storage") 
keyword_index = load_index_from_storage(storage_context1) # BM25 검색 인덱스 로드
vector_index = load_index_from_storage(storage_context2) # 밀집 검색 인덱스 로드

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_hybrid_search_keyword_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_hybrid_search_keyword_storage\index_store.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_hybrid_search_vertor_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch03_hybrid_search_vertor_storage\index_store.json.


In [8]:
## bm25 retriever 객체 생성(keyword_index 연결)
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(
    index=keyword_index,
    similarity_top_k=5,
)

# dense retriever 객체 생성(vector_index 연결)
semantic_retriever = vector_index.as_retriever(similarity_top_k=5)

resource module not available on Windows


c:\Users\USER\OneDrive\Desktop\Project\llamaindex_practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query='검색 증강 생성'
# 혼합 검색 클래스 객체 생성
hybrid = HybridSearch(bm25_retriever=bm25_retriever,
                      semantic_retriever=semantic_retriever)
combined_score = hybrid.retrieve(query=query)

In [10]:
# rufrhk 결과 출력
for node_score_obj in combined_score:
    print(node_score_obj)


Node ID: cf3078fc-e094-453f-90a0-098c0e1b154d
Text: 최신 검색 증강 시스템 연구
Score:  0.802

Node ID: 6eb47b4d-ada6-4b41-a6e4-d04efc9728af
Text: 최신 AI 논문: 검색 증강 기술
Score:  0.759

Node ID: 5f9fd9a7-b540-4a98-9a6a-7bf1df2a0d53
Text: RAG와 Dense Passage Retrieval의 비교 연구
Score:  0.396

Node ID: e15b2740-b07b-4ad6-9299-b0bd89791aac
Text: RAG을 활용한 챗봇 개발
Score:  0.410

Node ID: 49281b38-004a-4dcf-8b6e-9ea04fff9765
Text: RAG 기술의 최신 동향
Score:  0.403

